In [1]:
import json
import os
import sys
import cProfile

import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error

sys.path.append(r"../")
from AssistanceTransform import transform

# Performance
In this Notebook, I will be performance testing various parts of the AssistanceTransform library. This includes both speed and accuracy testing.

First getting the testing data ready:

In [2]:
image_coord_dict = dict()
image_coord_dict["00.jpg"] = np.array([[1470, 1430], [2333, 1355], [3247, 1781], [1935, 1952]])
image_coord_dict["01.jpg"] = np.array([[1495, 1552], [2219, 1589], [1843, 1969], [805, 1875]])
image_coord_dict["03.jpg"] = np.array([[1216, 1398], [2215, 1754], [3268, 1530], [2067, 1282]])   

def setup_vars():
    """Loads data for test_transform_image"""
    data_dir = r"./data/table"
    json_fp = os.path.join(data_dir, "anno.json")
    arr_fp = os.path.join(data_dir, "anno.npz")
    with open(json_fp, "r") as fp:
        mappings = json.load(fp)

    with np.load(arr_fp) as arrs:
        anno_dict = {img: {"heads": arrs[f"{prefix}heads"],
                           "feet": arrs[f"{prefix}feet"]}
                     for img, prefix in mappings.items()}
    
    for key, items in anno_dict.items():
        if key.endswith("02.jpg"):
            continue
        else:
            image_coords = image_coord_dict[key[-6:]]
        # feet and heads have been swapped in annotations
        reference = items["feet"], items["heads"]
        height = 0.095  # m
        STD = 0.01  # m
        img = Image.open(key)
        print(key)
        yield (img, reference, height, STD, image_coords)

## Speed

In [3]:
generator = setup_vars()
pr = cProfile.Profile()
pr.enable()
coords_sq = [transform.transform_image(*params, iters=1e5) for params in generator]
pr.disable()
pr.print_stats()

C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
  0%|                                               | 185/100000 [00:00<00:54, 1836.61it/s, acc_rate=0.08, factor=0.81]

D:\University\2020-2021\Internship\AssistanceTransform\notebooks\data\table\img_03.jpg


100%|█████████████████████████████████████████| 100000/100000 [01:06<00:00, 1496.76it/s, acc_rate=0.484, factor=0.0675]
C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
  0%|                                              | 158/100000 [00:00<01:03, 1578.47it/s, acc_rate=0.0547, factor=0.9]

       elevation_m   tilt_deg  heading_deg   roll_deg  probability
0         0.320169  82.372233   -59.817800  -0.888182    12.758389
1         0.315232  82.472352   -59.770878  -0.903301    12.665310
2         0.315232  82.472352   -59.770878  -0.903301    12.665310
3         0.315232  82.472352   -59.770878  -0.903301    12.665310
4         0.315232  82.472352   -59.770878  -0.903301    12.665310
...            ...        ...          ...        ...          ...
89994     0.783316  71.121371   -44.756902  19.645083    11.563897
89995     0.796160  70.987767   -44.906611  19.731363    11.557946
89996     0.796160  70.987767   -44.906611  19.731363    11.557946
89997     0.796160  70.987767   -44.906611  19.731363    11.557946
89998     0.796160  70.987767   -44.906611  19.731363    11.557946

[89999 rows x 5 columns]
D:\University\2020-2021\Internship\AssistanceTransform\notebooks\data\table\\img_00.jpg


100%|██████████████████████████████████████████| 100000/100000 [01:08<00:00, 1467.92it/s, acc_rate=0.391, factor=0.103]
C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
  0%|                                               | 163/100000 [00:00<01:01, 1623.80it/s, acc_rate=0.05, factor=0.81]

...

       15    0.000    0.000    0.013    0.001 _distn_infrastructure.py:779(__call__)
   300006    0.125    0.000    0.125    0.000 _distn_infrastructure.py:872(_argcheck)
   300021    0.129    0.000    0.129    0.000 _distn_infrastructure.py:884(_get_support)
   300006    2.815    0.000    7.591    0.000 _distn_infrastructure.py:905(_support_mask)
       36    0.000    0.000    0.000    0.000 _dtype.py:24(_kind_name)
       36    0.000    0.000    0.000    0.000 _dtype.py:307(_name_includes_bit_suffix)
       36    0.000    0.000    0.000    0.000 _dtype.py:321(_name_get)
   300006    3.147    0.000    8.089    0.000 _methods.py:143(_mean)
       18    0.000    0.000    0.000    0.000 _methods.py:37(_amax)
       45    0.000    0.000    0.000    0.000 _methods.py:53(_any)
        9    0.000    0.000    0.000    0.000 _methods.py:56(_all)
   300006    0.709    0.000    0.971    0.000 _methods.py:59(_count_reduce_items)
        3    0.000    0.000    0.000    0.000 _monitor.py:102(report

        6    0.000    0.000    0.000    0.000 concat.py:525(_combine_concat_plans)
        3    0.000    0.000    0.000    0.000 concat.py:534(_get_concat_axis)
       12    0.000    0.000    0.000    0.000 concat.py:550(_next_or_none)
        3    0.000    0.000    0.000    0.000 concat.py:567(<listcomp>)
        3    0.000    0.000    0.000    0.000 concat.py:584(_maybe_check_integrity)
        3    0.000    0.000    0.000    0.000 concat.py:591(_concat_indexes)
        3    0.000    0.000    0.000    0.000 concat.py:66(<listcomp>)
        3    0.000    0.000    0.002    0.001 concat.py:70(concat)
        6    0.000    0.000    0.000    0.000 concat.py:87(_get_mgr_concatenation_plan)
      189    0.000    0.000    0.001    0.000 config.py:104(_get_option)
        6    0.000    0.000    0.000    0.000 config.py:112(_set_option)
        6    0.000    0.000    0.000    0.000 config.py:176(get_default_val)
      186    0.000    0.000    0.001    0.000 config.py:232(__call__)
        3   

        3    0.000    0.000    0.000    0.000 function.py:48(__call__)
        3    0.000    0.000    0.000    0.000 function.py:86(process_skipna)
   599114    0.117    0.000    0.117    0.000 function_base.py:1641(_extract_dispatcher)
   599114    1.601    0.000   11.176    0.000 function_base.py:1645(extract)
   299557    0.064    0.000    0.064    0.000 function_base.py:1697(_place_dispatcher)
   299557    0.273    0.000    0.912    0.000 function_base.py:1701(place)
       60    0.000    0.000    0.000    0.000 function_base.py:2048(__init__)
     1275    0.000    0.000    0.001    0.000 generic.py:10(_check)
        3    0.000    0.000    0.000    0.000 generic.py:1670(__iter__)
       36    0.000    0.000    0.000    0.000 generic.py:195(__init__)
        6    0.000    0.000    0.000    0.000 generic.py:232(attrs)
        6    0.000    0.000    0.000    0.000 generic.py:249(_validate_dtype)
       18    0.000    0.000    0.000    0.000 generic.py:3177(_set_as_cached)
        3  

        6    0.000    0.000    0.000    0.000 printing.py:29(adjoin)
        6    0.000    0.000    0.000    0.000 printing.py:50(<listcomp>)
       90    0.000    0.000    0.000    0.000 printing.py:64(justify)
       30    0.000    0.000    0.000    0.000 printing.py:69(<listcomp>)
       60    0.000    0.000    0.000    0.000 printing.py:73(<listcomp>)
   900021   12.349    0.000   27.165    0.000 projection.py:335(getRay)
        3    0.000    0.000    0.000    0.000 projection.py:89(__init__)
        8    0.000    0.000    0.000    0.000 py3k.py:53(isfileobj)
        8    0.000    0.000    0.000    0.000 random.py:224(_randbelow)
        8    0.000    0.000    0.000    0.000 random.py:256(choice)
        1    0.000    0.000    0.000    0.000 random.py:88(__init__)
        1    0.000    0.000    0.000    0.000 random.py:97(seed)
        9    0.000    0.000    0.000    0.000 range.py:134(_simple_new)
        6    0.000    0.000    0.000    0.000 range.py:153(_data)
        6    0.00

      333    0.000    0.000    0.000    0.000 {built-in method _struct.unpack}
     5053    0.021    0.000    0.021    0.000 {built-in method _thread.allocate_lock}
       27    0.000    0.000    0.000    0.000 {built-in method _thread.get_ident}
        3    0.000    0.000    0.000    0.000 {built-in method _thread.start_new_thread}
       12    0.000    0.000    0.000    0.000 {built-in method _warnings._filters_mutated}
        1    0.000    0.000    0.000    0.000 {built-in method _weakref.proxy}
        3    0.000    0.000    0.000    0.000 {built-in method atexit.register}
       24    0.000    0.000    0.001    0.000 {built-in method builtins.__build_class__}
      6/5    0.000    0.000    0.008    0.002 {built-in method builtins.__import__}
     2531    0.002    0.000    0.002    0.000 {built-in method builtins.abs}
       66    0.000    0.000    0.000    0.000 {built-in method builtins.all}
       48    0.000    0.000    0.000    0.000 {built-in method builtins.any}
      185 

## Accuracy
In order to figure out how close CameraTransform is to "real" positions, we have to take a derivative of these positions. For the purposes of this assignment, I've chosen the area of the polygon, as this is an important variable in the use case.

In [4]:
def area(poly):
    x, y = poly[:,:2].T
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

In [5]:
est_areas = np.array([area(poly) for poly in coords_sq])
real = np.repeat(1.33455, est_areas.size)

In [6]:
est_areas

array([1.63628569, 1.56604306, 1.62892857])

In [7]:
mean_squared_error(real, est_areas, squared=False)

0.2776641880599487